In [1]:
pip install langchain_community

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.llms import Ollama

In [3]:
llm = Ollama(model="llama2")

In [4]:
llm.invoke("Can you add an appropriate category next to each of the following expenses. Respond with a list of categories separated by commas. For example, Spotify AB by Adyen - \
Entertainment, Beta Boulders Ams Amsterdam Nld - Sports, etc.: \
Taxi Utrecht, Ministerie van Justitie en Veiligheid, Etos AMSTERDAM NLD, Bistro Bar Amsterdam")

"\nOf course! I'd be happy to help you categorize your expenses. Here are the categories for each of the expenses you provided:\n\nTaxi Utrecht - Transportation\nMinisterie van Justitie en Veiligheid - Government\nEtos AMSTERDAM NLD - Food and Beverage\nBistro Bar Amsterdam - Entertainment\n\nI hope this helps! Let me know if you need any further assistance."

In [5]:
pip install pandas

In [15]:
# Read the transactions_2022_2023.csv file 
import pandas as pd
df = pd.read_csv("transactions_2023_2024.csv")
df.head()

,Date,Name / Description,Expense/Income,Amount
0,07-31-2024,AplPay APPLE.COM/BILINTERNET CHARGE CA,Expense,30.21
1,07-31-2024,BH* BETTERHELP MOUNTAIN VIEW CA,Expense,360.00
2,07-31-2024,LYFT 855-280-0278 CA,Expense,25.84
3,07-31-2024,LYFT 855-280-0278 CA,Expense,33.95
4,07-30-2024,AplPay STANZA-VINYL San Francisco CA,Expense,9.75


In [16]:
# Get unique transactions in the Name / Description column
unique_transactions = df["Name / Description"].unique()
len(unique_transactions)

63

In [17]:
unique_transactions[1:10]

array(['BH* BETTERHELP MOUNTAIN VIEW CA', 'LYFT 855-280-0278 CA',
       'AplPay STANZA-VINYL San Francisco CA',
       'AplPay HUMPHRY SLOCO San Francisco CA',
       'AplPay WHOLEFDS STN SAN FRANCISCO CA',
       'AplPay WALGREENS SAN FRANCISCO CA',
       'AplPay COWORKING San Francisco CA',
       'AplPay TST* THE ITAL SAN FRANCISCO CA',
       "AplPay TRADER JOE'S SAN FRANCISCO CA"], dtype=object)

In [18]:
# Get index list
#https://stackoverflow.com/questions/47518609/for-loop-range-and-interval-how-to-include-last-step
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i
    yield stop

index_list = list(hop(0, len(unique_transactions), 30))
index_list

[0, 30, 60, 63]

In [21]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df 

In [22]:
# Intialise the categories_df_all dataframe
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['Sure! Here are the expenses with appropriate categories:', '', '1. AplPay APPLE.COM/BILINTERNET CHARGE CA - Technology', '2. BH* BETTERHELP MOUNTAIN VIEW CA - Health and Wellness', '3. LYFT 855-280-0278 CA - Transportation', '4. AplPay STANZA-VINYL San Francisco CA - Music and Entertainment', '5. AplPay HUMPHRY SLOCO San Francisco CA - Food and Beverage', '6. AplPay WHOLEFDS STN SAN FRANCISCO CA - Grocery', '7. AplPay WALGREENS SAN FRANCISCO CA - Pharmacy', '8. AplPay COWORKING San Francisco CA - Office Supplies', '9. TST* THE ITAL SAN FRANCISCO CA - Restaurant', "10. TRADER JOE'S SAN FRANCISCO CA - Grocery", '11. BT*DD *DOORDASH SAN FRANCISCO CA - Food Delivery', '12. COWORKING SAN FRANCISCO CA - Office Supplies', '13. NAME-CHEAP.COM* IJTI PHOENIX AZ - Online Shopping', '14. DD *DOORDASH DASHPASS SAN FRANCISCO CA - Food Delivery', '15. JOE & THE JUICE SAN FRANCISCO CA - Food and Beverage', '16. NETFLIX.COM 866-579-7172 CA - Entertainment', '17. LIME*PASS SAN FRANCISCO CA - Transport

ValueError: Columns must be same length as key

In [23]:
categories_df_all

,Transaction vs category,Transaction,Category
0,Sure! Here are the expenses with appropriate c...,Sure! Here are the expenses with appropriate c...,None
1,,,None
2,1. AplPay APPLE.COM/BILINTERNET CHARGE CA - Te...,1. AplPay APPLE.COM/BILINTERNET CHARGE CA,Technology
3,2. BH* BETTERHELP MOUNTAIN VIEW CA - Health an...,2. BH* BETTERHELP MOUNTAIN VIEW CA,Health and Wellness
4,3. LYFT 855-280-0278 CA - Transportation,3. LYFT 855-280-0278 CA,Transportation
5,4. AplPay STANZA-VINYL San Francisco CA - Musi...,4. AplPay STANZA-VINYL San Francisco CA,Music and Entertainment
6,5. AplPay HUMPHRY SLOCO San Francisco CA - Foo...,5. AplPay HUMPHRY SLOCO San Francisco CA,Food and Beverage
7,6. AplPay WHOLEFDS STN SAN FRANCISCO CA - Grocery,6. AplPay WHOLEFDS STN SAN FRANCISCO CA,Grocery
8,7. AplPay WALGREENS SAN FRANCISCO CA - Pharmacy,7. AplPay WALGREENS SAN FRANCISCO CA,Pharmacy
9,8. AplPay COWORKING San Francisco CA - Office ...,8. AplPay COWORKING San Francisco CA,Office Supplies


In [24]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array([None, 'Technology', 'Health and Wellness', 'Transportation',
       'Music and Entertainment', 'Food and Beverage', 'Grocery',
       'Pharmacy', 'Office Supplies', 'Restaurant', 'Food Delivery',
       'Online Shopping', 'Entertainment', 'Clothing and Accessories',
       'Retail', 'Beauty and Personal Care', 'Travel'], dtype=object)

In [25]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

In [26]:
# Remove the numbering eg "1. " from Transaction column
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

C:\Users\ep\AppData\Local\Temp\ipykernel_6448\2440992137.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')


,Transaction vs category,Transaction,Category
2,1. AplPay APPLE.COM/BILINTERNET CHARGE CA - Te...,1. AplPay APPLE.COM/BILINTERNET CHARGE CA,Technology
3,2. BH* BETTERHELP MOUNTAIN VIEW CA - Health an...,2. BH* BETTERHELP MOUNTAIN VIEW CA,Health and Wellness
4,3. LYFT 855-280-0278 CA - Transportation,3. LYFT 855-280-0278 CA,Transportation
5,4. AplPay STANZA-VINYL San Francisco CA - Musi...,4. AplPay STANZA-VINYL San Francisco CA,Music and Entertainment
6,5. AplPay HUMPHRY SLOCO San Francisco CA - Foo...,5. AplPay HUMPHRY SLOCO San Francisco CA,Food and Beverage
7,6. AplPay WHOLEFDS STN SAN FRANCISCO CA - Grocery,6. AplPay WHOLEFDS STN SAN FRANCISCO CA,Grocery
8,7. AplPay WALGREENS SAN FRANCISCO CA - Pharmacy,7. AplPay WALGREENS SAN FRANCISCO CA,Pharmacy
9,8. AplPay COWORKING San Francisco CA - Office ...,8. AplPay COWORKING San Francisco CA,Office Supplies
10,9. TST* THE ITAL SAN FRANCISCO CA - Restaurant,9. TST* THE ITAL SAN FRANCISCO CA,Restaurant
11,10. TRADER JOE'S SAN FRANCISCO CA - Grocery,10. TRADER JOE'S SAN FRANCISCO CA,Grocery


In [27]:
# Merge the categories_df_all with the transactions_2023_2024.csv dataframe (df)
df = pd.read_csv("transactions_2023_2024.csv")
# Reset the index of both DataFrames to ensure alignment by index
df.reset_index(drop=True, inplace=True)
categories_df_all.reset_index(drop=True, inplace=True)

# Concatenate DataFrames horizontally (axis=1)
df_combined = pd.concat([df, categories_df_all], axis=1)

# Save the combined DataFrame to a new CSV file if needed
df_combined.to_csv("combined_transactions_categories.csv", index=False)

# Display the first few rows of the combined DataFrame
print(df_combined.head())

         Date                      Name / Description Expense/Income  Amount  \
0  07-31-2024  AplPay APPLE.COM/BILINTERNET CHARGE CA        Expense   30.21   
1  07-31-2024         BH* BETTERHELP MOUNTAIN VIEW CA        Expense  360.00   
2  07-31-2024                    LYFT 855-280-0278 CA        Expense   25.84   
3  07-31-2024                    LYFT 855-280-0278 CA        Expense   33.95   
4  07-30-2024    AplPay STANZA-VINYL San Francisco CA        Expense    9.75   

                             Transaction vs category  \
0  1. AplPay APPLE.COM/BILINTERNET CHARGE CA - Te...   
1  2. BH* BETTERHELP MOUNTAIN VIEW CA - Health an...   
2           3. LYFT 855-280-0278 CA - Transportation   
3  4. AplPay STANZA-VINYL San Francisco CA - Musi...   
4  5. AplPay HUMPHRY SLOCO San Francisco CA - Foo...   

                                 Transaction                 Category  
0  1. AplPay APPLE.COM/BILINTERNET CHARGE CA               Technology  
1         2. BH* BETTERHELP MOUNTAIN V